<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_share_cirlce.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# 3.4 Data Sharing
Science is much more impactful once it's shared. Therefore, we are going to learn how to 
open up our scientific output from a cloud platform, so that is openly available - and 
has the chance to make the impact it should.
- Reuse the workflow we have used before for creating the snow covered area
- Select AOI,
- Recreate process graph, 
- Download results for one time-step
  - A Snow Cover Area map in the COG format
  - A STAC metadata item that is provided with the result from openEO at CDSE
- Adapt the STAC item
- Upload the results and make them available openly via a STAC browser and web map


## Libraries

In [1]:
!cp -r $DATA_PATH/33_results/ $HOME/

In [15]:
import json
import os
import subprocess
from datetime import datetime

import openeo
import numpy as np
import leafmap
import geopandas as gpd
import shapely
from shapely.geometry import Polygon

import rioxarray as rio
import xarray

Import utility functions

In [16]:
%run 33_cubes_utilities.py

## Login

Connect to the copernicus dataspace ecosystem.

In [17]:
conn = openeo.connect('https://openeo.dataspace.copernicus.eu/')

Authenticate login

In [18]:
conn.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

Check if the login worked

In [ ]:
conn.describe_account()

## Select an Area of Interest and Time Frame
Start by selecting a center point of the area you would like to analyse from the map shown below. The starting extent is the full alps. Zoom in to an area and choose a region that has not been mapped yet. Make sure not to overlap too much with already mapped areas. It's a community mapping project :)
Create a 1 km bounding box around it. This will be the area you are calculating the snow covered area for. 

Execute the cell below to show the map. Zoom to a location you want to analyze. Use the location symbol to select a point. A marker appears on the map. This is the center of your area of interest.

In [20]:
m = leafmap.Map(center=(47.005, 11.507), zoom=7.5)
m

Map(center=[47.005, 11.507], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

Now this cell will get the coordinates of the marker you have placed.

In [21]:
feat = m.draw_features
geom = feat[0]['geometry']['coordinates']

IndexError: list index out of range

This will create a 1 km bounding box around the chosen location. And visualize it.

In [ ]:
distance_km = 1
# Create a bounding box around the point
bbox = create_bounding_box(geom[0], geom[1], distance_km)
visualize_bbox(m, bbox)

Now we'll select the time frame. We'll start with the winter months of 2023. 

In [ ]:
temporal_extent = ["2023-02-01", "2023-06-01"]

## Reuse the process graph of the snow covered area data cube
We've saved the python code that we had used to create the snow cover area data cube into a python function `calculate_sca()`. It's stored in `cubes_utilities.py`. It creates a 4 dimensional data cube with the dimensions: x, y, time, bands.
As parameters we have exposed the bounding box and temporal extent. We will update them with the choices we have made above. 

In [ ]:
snow_map_4dcube = calculate_sca(conn, bbox, temporal_extent)
snow_map_4dcube

## Reduce the time dimension
We want to calculate the SCA for the winter period of a given year. Therefore, we need to reduce the values along the time dimension. We'll use the process `reduce_dimension()` with a `median()` to accomplish this. We are directly continuing to build on our process graph that we have loaded above.

In [ ]:
snow_map_3dcube = snow_map_4dcube.reduce_dimension(reducer="median", dimension="t")
snow_map_3dcube

## Download result
To finish our process graph we add the `save_result()` process choosing the `GTiff` format. It creates a COG out of the box with openEO on CDSE.

In [ ]:
# create a batch job
snowmap_cog = snow_map_3dcube.save_result(format = "GTiff") #, options = {"overviews": "AUTO"})

We register the job as a batch job on the backend and start the processing.

In [ ]:
job = snowmap_cog.create_job(title="snowmap_cog")
job.start_and_wait()

Now let's wait until the job is finished and then download the results.

In [ ]:
if job.status() == "finished":
    results = job.get_results()
    results.download_files("33_results/")

## Load results
Now we can open the COG and visualize it.

In [ ]:
snowmap = rio.open_rasterio("33_results/openEO.tif",decode_coords="all")
snowmap

In [ ]:
snowmap.plot()

## Load STAC metadata
In addition to the COG we also receive STAC metadata for our result.
Let's have a look at it.

In [ ]:
stac_collection = results.get_metadata()

In [ ]:
stac_collection

### Adding Author of the data

Add your information to become visible as author of the data -  description of each field can be found here: https://github.com/radiantearth/stac-spec/blob/master/item-spec/common-metadata.md#provider-object

Please note that leaving the field empty will lead to failed validation of STAC item

In [ ]:
author = [{
    "name": "",
    "description": "",
    "roles": ["processor"],
}]

providers = stac_collection["providers"] + author

author_id = [nam[:2] for nam in author[0]["name"].split(" ")]

# generate timestamp
# ts = datetime.timestamp(datetime.now())
ts = datetime.now().timestamp()

Creating actual polygon geometry from previously created bbox information

In [ ]:
min_x, min_y, max_x, max_y = stac_collection["extent"]["spatial"]["bbox"][0]
geometry = {
    "type": "Polygon",
    "coordinates": [[
        [min_x, min_y],
        [max_x, min_y],
        [max_x, max_y],
        [min_x, max_y],
        [min_x, min_y]
    ]]
}


Let's create the actual STAC item describing your data! As talked about in previous lessons, STAC item has various required fields which need to be present and filled correctly. For the field ID we assign the fixed name snowcover and the initials of your name. That will be visible on the STAC browser once you have submitted the result!

In [ ]:
stac_item = {
    "type": "Feature", 
    "stac_version": stac_collection["stac_version"],
    "stac_extensions": [],
    "id": "snowcover_" + "".join(author_id).lower()+ "_" + str(ts),
    "geometry": geometry,
    "bbox": stac_collection["extent"]["spatial"]["bbox"][0],
    
    "properties": {
       "datetime": None, 
        "start_datetime": stac_collection["extent"]["temporal"]["interval"][0][0],
        "end_datetime": stac_collection["extent"]["temporal"]["interval"][0][1],
        "providers" : providers
                 },
    
    "links": stac_collection["links"],
    "assets": {"visual": {
      "href": "openEO.tif",
      "type": "image/tiff; application=geotiff; profile=cloud-optimized",
      "title": "Snow coverage",
      "roles": [
        "data"
              ]
            }
        },
}

In [ ]:
stac_item

Saving the resulting item as stac_item.json into results folder

In [ ]:
stac_json = json.dumps(stac_item)
with open("33_results/stac_item.json", "w") as file:
    file.write(stac_json)

Validating that STAC item is important - non valid STAC will not be displayed in the STAC browser after upload

In [ ]:
from stac_validator import stac_validator
import requests
stac = stac_validator.StacValidate()
f = open('33_results/stac_item.json')
data = json.load(f)
stac.validate_dict(data)
print(stac.message)

### Now it is time to upload solution to the submission folder and make results visible in STAC browser

Upload both STAC json file and final .tif file to "submissions" folder in your home directory

You can use code below or copy the code for you - if your results are stored at different path, you need to modify the path correctly

In [ ]:
!cp ./33_results/stac_item.json ~/submissions/
!cp ./33_results/openEO.tif ~/submissions/

And now by executing the cell bellow, update of the STAC browser will start. 

In [ ]:
env_var1 = os.getenv('EMAIL')
curl_command = f"curl -X POST -F token=glptt-42d31ac6f592a9e321d0e4877e654dc50dcf4854 -F ref=main -F 'variables[USER_DIRECTORY]=\"{env_var1}\"' https://gitlab.eox.at/api/v4/projects/554/trigger/pipeline" 
process = subprocess.Popen(curl_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

### Your results are online!
You can now browse your results together with all the other submissions at the publicly available STAC Catalog! You can check your snow cover map, that you are correctly listed as the author and that your contribution has the correct name. The license on the STAC Collection "Cubes and Clouds: Snow Cover" is CC-BY-4.0. The STAC Collection also has it's own DOI XXX. 

Congratulations you have just contributed to a community mapping project that is completely open source, open data and FAIR! Make sure to show it also to your friends, colleagues or potential employers :)

https://esa.pages.eox.at/cubes-and-clouds-catalog/browser/#/?.language=en

If you would like to redo your submission, you can still update your files in submissions folder and once ready, run again the code in the cell above

Happy coding!